<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/082_FromPlanning_ToReflection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The key elements you should focus on in this notebook are:

1. **Shift from Planning to Reflection**

   * The earlier `PlanFirstCapability` was about forward planning before actions.
   * `ProgressTrackingCapability` focuses on looking back at *what just happened* after each loop iteration.
   * This mirrors the human process: **plan → act → evaluate → adjust**.

2. **Track Progress Tool**

   * It’s similar to the `create_plan` tool, but instead of asking *“What should we do?”*, it asks *“What have we done, what’s left, and what’s blocking us?”*
   * This helps the agent avoid going blindly forward without reassessment.

3. **Integration into the Agent Loop**

   * The hook `end_agent_loop()` is used so progress tracking happens after each iteration.
   * This hook ensures that the **most recent results are included in the reflection**, rather than relying on older assumptions.

4. **Frequency Control (`track_frequency`)**

   * You can set how often progress tracking runs (e.g., every iteration, every 2nd iteration, etc.).
   * This lets you balance between reflection quality and efficiency.

5. **Benefits for Long-Running Tasks**

   * Maintains focus on goals across multiple steps.
   * Creates an audit trail of reasoning and actions.
   * Helps identify blockers early.
   * Avoids repeating work or drifting off-task.

Focus on **the difference between pre-action planning and post-action reflection**—because together, they form a full feedback loop that makes agents more autonomous and adaptable.






# 🎯 Tracking Progress: Turning Agents into Thoughtful Problem-Solvers

When tackling **complex, multi-step tasks**, agents can’t just charge forward blindly—they need to pause, reflect, and adapt.

Enter **progress tracking**: a capability that adds **reflection** to the end of every agent loop iteration. By stopping to assess **what just happened**, the agent gains clarity on its current position and what needs to happen next.

---

## 🔍 Why It Matters

Without reflection, an agent risks:

* **Repeating work** it’s already done
* **Drifting** away from its original goal
* **Missing blockers** that could derail progress later

With progress tracking, we give the agent the superpower of **self-awareness** during execution.

---

## 🛠 How It Works

We introduce a function called `track_progress()` that **evaluates the agent’s current state** after each action.
It’s like our earlier planning function—but instead of *“What should I do?”*, it asks:

> 💭 *“What did I just do, what’s left, and what’s in my way?”*

Here’s what it considers:

* **Available tools** for the next step
* **Memory context** from earlier steps
* **Completed actions** so far
* **Any blockers or issues** that have appeared

---

## 🔄 The Continuous Feedback Loop

By running `track_progress()` at the **end** of each loop iteration:

1. ✅ The agent **confirms progress** on completed steps
2. 🚧 Identifies **blockers early** before they cause failure
3. 🧭 Dynamically **adapts the plan** based on real-time feedback
4. 📈 Improves **overall efficiency** for complex workflows

---

## 🧠 The Human Parallel

This mirrors how **humans** tackle problems:

1. **Plan** → Decide what needs to be done
2. **Act** → Execute a step
3. **Evaluate** → See what happened
4. **Adjust** → Modify the approach if needed

---

💡 **Bottom line:** Adding reflection transforms your agent from a **mindless executor** into a **thoughtful problem-solver**, capable of navigating complex tasks with strategy and precision.



# Progress Tracking Tool

A few notes to focus on:

* **Dependency injection:** `_memory` (notice the underscore) and `action_registry` are injected by your Environment. The agent only “sees” the tool name and explicit args (none here besides injected ones).
* **Context curation:** You’re filtering memory to `['user','system']`, which keeps the prompt lean and safer. Nice.
* **Tool cataloging:** It pulls `action_registry.get_actions()` to list available tools—great for suggesting next steps.

If you want to tighten it up even more (optional polish):

* Return **structured JSON** (e.g., `{ progress, blockers, next_steps, suggested_tools }`) using your `prompt_llm_for_json` pattern. That’ll make downstream use easier.
* Add a **cap** on how much memory you include (e.g., last N items) to control token usage.
* Consider a **`track_progress` tag** (you already used `"prompts"`) if you’re grouping analytics/ops tools separately.



In [ ]:
@register_tool(tags=["prompts"])
def track_progress(action_context: ActionContext,
                   _memory: Memory,
                   action_registry: ActionRegistry) -> str:
    """Generate a progress report based on the current task, available tools, and memory context."""

    # Get tool descriptions for the prompt
    tool_descriptions = "\n".join(
        f"- {action.name}: {action.description}"
        for action in action_registry.get_actions()
    )

    # Get relevant memory content
    memory_content = "\n".join(
        f"{m['type']}: {m['content']}"
        for m in _memory.items
        if m['type'] in ['user', 'system']
    )

    # Construct the prompt as a string
    prompt = f"""Given the current task and available tools, generate a progress report.

Think through this step by step:

1. Identify the key components of the task and the intended outcome.
2. Assess the progress made so far based on available information.
3. Identify any blockers or issues preventing completion.
4. Suggest the next steps to move forward efficiently.
5. Recommend any tool usage that might help complete the task.

Write your progress report in clear, structured points.

Available tools:
{tool_descriptions}

Task context from memory:
{memory_content}

Provide a well-organized report on the current progress and next steps."""

    return prompt_llm(action_context=action_context, prompt=prompt)

# Progress Tracking Capability — What it does & why it matters

**Goal:** add a lightweight “reflect → adjust” step after each tool action so the agent doesn’t just *do*, it also *thinks about what it did* and what to do next.

## What this capability adds

* **End-of-loop reflection:** It runs **after each iteration** of the agent loop (or every N iterations) and writes a concise progress report into memory.
* **Context-aware summaries:** The report is generated using:

  * the **current memory** (task context, prior steps), and
  * the **action registry** (which tools are available next),
    so suggestions are realistic and grounded in what the agent can actually do.
* **Persistent breadcrumbs:** Each report is stored in memory, creating an **audit trail** of what happened, blockers, and next steps.

## The key knobs to notice

* `track_frequency`: controls **how often** to run reflection.

  * `1` = every loop (max insight, more tokens/latency)
  * `2+` = periodic check-ins (balanced cost)
* `memory_type`: where the report lands (e.g., `"system"` to increase its weight in the next prompt, or `"assistant"` if you want it treated as normal assistant output).
* Internal counter (`iteration_count`): ensures **deterministic cadence** for reflection (e.g., “every 3rd loop”).

## Why it runs at the *end* of the loop

* You reflect with **fresh results** from the just-executed action.
* You can **spot blockers early** (API failure, missing data) and immediately course-correct.
* It avoids re-planning on stale assumptions from before the action ran.

## Trade-offs (be intentional)

* **Cost/latency:** Each reflection is an extra LLM call. Use `track_frequency` to tune.
* **Context size:** Reports add tokens to memory. Consider **summarizing or pruning** old reports over time.
* **Signal quality:** If everything is already simple and linear, this may be overkill; for complex, branching tasks it’s gold.

## How it plays with other capabilities

* **Plan-First + Progress-Tracking** = plan → act → reflect → adjust.
  The plan sets direction; progress tracking keeps it **adaptive**.
* Works well with **Selective Memory Sharing**: share only the latest progress report when delegating to another agent.
* Pairs nicely with **Safety/Staging**: reflect on risky steps and propose safer alternatives.

## Practical tips

* **Structure the output**: have `track_progress` return JSON (`progress`, `blockers`, `next_steps`, `suggested_tools`). Easier to parse in the next loop.
* **Throttle on success**: if `blockers` is empty for a few iterations, temporarily **increase `track_frequency`** to save cost.
* **Tag memories**: prefix content (e.g., `"[Progress]"`) so you can filter them quickly for prompts or dashboards.
* **Metrics hooks**: optionally emit counters (iterations, blockers found) for observability.

**Bottom line:** This capability gives your agent a simple, composable “meta-cognition” loop—small overhead, big gains in reliability, debuggability, and outcome quality, especially on multi-step or long-running tasks.


In [ ]:
class ProgressTrackingCapability(Capability):
    def __init__(self, memory_type="system", track_frequency=1):
        super().__init__(
            name="Progress Tracking",
            description="Tracks progress and enables reflection after actions"
        )
        self.memory_type = memory_type
        self.track_frequency = track_frequency
        self.iteration_count = 0

    def end_agent_loop(self, agent, action_context: ActionContext):
        """Generate and store progress report at the end of each iteration."""
        self.iteration_count += 1

        # Only track progress on specified iterations
        if self.iteration_count % self.track_frequency != 0:
            return

        # Get the memory and action registry from context
        memory = action_context.get_memory()
        action_registry = action_context.get_action_registry()

        # Generate progress report
        progress_report = track_progress(
            action_context=action_context,
            _memory=memory,
            action_registry=action_registry
        )

        # Add the progress report to memory
        memory.add_memory({
            "type": self.memory_type,
            "content": f"Progress Report (Iteration {self.iteration_count}):\n{progress_report}"
        })

In [ ]:
# This capability uses the track_progress tool to generate detailed progress reports.
# Let’s see how it transforms agent behavior in practice:

# Create an agent with progress tracking
agent = Agent(
    goals=[
        Goal(
            name="data_processing",
            description="Process and analyze customer feedback data"
        )
    ],
    capabilities=[
        ProgressTrackingCapability(track_frequency=2)  # Track every 2nd iteration
    ],
    # ... other agent configuration
)

# Example execution flow
memory = agent.run("Analyze customer feedback from Q4 and identify top issues")
# After each iteration (or every N iterations), the agent will pause to reflect.

## Reflect -> Adapt

Because it works *after* a tool executes, the agent is looking at **real-world results**, not just its predictions:

* If the tool failed (API timeout, bad credentials), the reflection will flag a **blocker** right away.
* If a step finished but uncovered new information, the reflection can recommend **adjusting the sequence** or adding new steps.
* If everything’s going smoothly, it simply confirms progress and moves to the next planned action.

It’s a lot like how a human would work on a multi-step task:

1. Try something.
2. Look at the outcome.
3. Ask “Is this working? Do I need to change course?”
4. If yes, adjust the plan before moving forward.

Because the progress report goes into **memory**, the **next loop’s prompt** already contains the updated situation, so the LLM naturally incorporates that into its next decision — without you hardcoding any new branching logic.

If you combine this with a **Plan-First** capability, you basically have:

> Plan → Act → Reflect → Adapt → Repeat

…which is about as close as you can get to giving an agent *situational awareness* without manually micro-managing every step.






## ✅ Benefits of End-of-Loop Progress Tracking

Tracking progress **at the end** of each loop iteration — rather than at the beginning — offers key advantages:

* **Assess the real impact** of the most recent action.
* **Base decisions on fresh data** in memory, not outdated assumptions from the original plan.
* **Adapt strategy dynamically** based on actual results.
* **Create a clear audit trail** of agent decision-making for transparency and debugging.

---

## 📊 Using Progress Reports in Decision Making

Stored progress reports become part of the **agent’s memory**, shaping its future decisions.
When the agent chooses its next action, it can reference these reports to:

* ✅ Avoid **repeating completed steps**.
* 🚧 Address **blockers** that were identified.
* 📅 Follow through on **recommended next steps**.
* 🛠 Use **suggested tools** effectively.

---

## 🎯 Why This Matters for Long-Running Tasks

The combination of **immediate reflection** + **persistent memory** helps the agent:

* Stay focused on its **goals**.
* Adapt to **new information** and **changing circumstances**.
* Spot when things are **going wrong** — and course-correct early.

This is especially valuable for **complex or long-running workflows**, where **maintaining context** is crucial for success.

---

This section basically formalizes the *"plan → act → reflect → adapt"* cycle we talked about earlier — except here the reflection is **baked into the loop** so it happens automatically every time.
